In [2]:
import torch
import torch.nn as nn

# Bi-GRU 모델 정의
class BiGRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(BiGRU, self).__init__()
        self.bigru = nn.GRU(input_size, hidden_size, num_layers, 
                           batch_first=True, bidirectional=True)
    
    def forward(self, x):
        output, hidden = self.bigru(x)
        return output, hidden

# 차원 변화 확인
input_size = 100  # 입력 특성 차원
hidden_size = 256  # 은닉 상태 차원
seq_length = 20   # 시퀀스 길이
batch_size = 32   # 배치 크기

# 모델 생성
model = BiGRU(input_size, hidden_size)

# 더미 입력 데이터 생성
input_data = torch.randn(batch_size, seq_length, input_size)

print(f"입력 차원: {input_data.shape}")

# Bi-GRU를 통과시켜 representation 추출
with torch.no_grad():
    output, hidden = model(input_data)

print(f"출력 차원: {output.shape}")
print(f"은닉 상태 차원: {hidden.shape}")

# 차원 변화 설명
print("\n=== 차원 변화 분석 ===")
print(f"입력: (batch_size={batch_size}, seq_length={seq_length}, input_size={input_size})")
print(f"출력: (batch_size={batch_size}, seq_length={seq_length}, hidden_size*2={hidden_size*2})")
print(f"은닉상태: (num_layers*2={2}, batch_size={batch_size}, hidden_size={hidden_size})")
print("\n※ Bidirectional이므로 출력 차원은 hidden_size의 2배가 됩니다.")

# hidden과 output의 차이점 분석
print("=== Hidden vs Output 차이점 ===")
print(f"Output 형태: {output.shape}")  # (batch_size, seq_length, hidden_size*2)
print(f"Hidden 형태: {hidden.shape}")  # (num_layers*2, batch_size, hidden_size)

print("\n=== 각각의 의미 ===")
print("Output: 모든 시간 스텝에서의 은닉 상태 (forward + backward 결합)")
print("Hidden: 마지막 시간 스텝의 은닉 상태 (forward와 backward 분리)")

# Hidden state 분석
forward_hidden = hidden[0]  # Forward GRU의 마지막 은닉 상태
backward_hidden = hidden[1]  # Backward GRU의 마지막 은닉 상태

print(f"\nForward hidden: {forward_hidden.shape}")
print(f"Backward hidden: {backward_hidden.shape}")

# Output의 마지막 시간 스텝과 Hidden 비교
last_output = output[:, -1, :]  # 마지막 시간 스텝의 output
print(f"\nLast output: {last_output.shape}")

# Forward와 Backward 분리
last_forward = output[:, -1, :hidden_size]   # Forward 부분
last_backward = output[:, -1, hidden_size:]  # Backward 부분

print(f"Last forward from output: {last_forward.shape}")
print(f"Last backward from output: {last_backward.shape}")

# Decoder에서 사용할 때의 권장사항
print("\n=== Decoder에서 사용 권장사항 ===")
print("1. Sequence-to-sequence: hidden state 사용 (context vector로)")
print("2. Attention mechanism: output 사용 (모든 시간 스텝 정보)")
print("3. Classification: output의 마지막 또는 평균 사용")

# Context vector 생성 예시
context_vector = torch.cat([forward_hidden, backward_hidden], dim=1)
print(f"\nContext vector for decoder: {context_vector.shape}")


입력 차원: torch.Size([32, 20, 100])
출력 차원: torch.Size([32, 20, 512])
은닉 상태 차원: torch.Size([2, 32, 256])

=== 차원 변화 분석 ===
입력: (batch_size=32, seq_length=20, input_size=100)
출력: (batch_size=32, seq_length=20, hidden_size*2=512)
은닉상태: (num_layers*2=2, batch_size=32, hidden_size=256)

※ Bidirectional이므로 출력 차원은 hidden_size의 2배가 됩니다.
=== Hidden vs Output 차이점 ===
Output 형태: torch.Size([32, 20, 512])
Hidden 형태: torch.Size([2, 32, 256])

=== 각각의 의미 ===
Output: 모든 시간 스텝에서의 은닉 상태 (forward + backward 결합)
Hidden: 마지막 시간 스텝의 은닉 상태 (forward와 backward 분리)

Forward hidden: torch.Size([32, 256])
Backward hidden: torch.Size([32, 256])

Last output: torch.Size([32, 512])
Last forward from output: torch.Size([32, 256])
Last backward from output: torch.Size([32, 256])

=== Decoder에서 사용 권장사항 ===
1. Sequence-to-sequence: hidden state 사용 (context vector로)
2. Attention mechanism: output 사용 (모든 시간 스텝 정보)
3. Classification: output의 마지막 또는 평균 사용

Context vector for decoder: torch.Size([32, 512])
